In [72]:
#Python Packages
import pandas as pd
import numpy as np
import xarray as xr
import glob as gb 
import csv
import itertools
from pathlib import Path
from pandas import *
import os.path
import matplotlib.pyplot as plt
import cv2

In [73]:
#Load in functions from freelymoving ephys
import sys
sys.path.insert(0, 'C:/Users/nlab/Documents/GitHub/FreelyMovingEphys')
from utils.paths import find
from utils.time import open_time1
from utils.time import open_time
from utils.format_data import open_h5
#from utils.format_data import open_ma_h5

#function to get correct frame in case there's lag in the acquisition
def find_first(item, vec):
    return np.argmin(np.abs(vec-item))

In [74]:
##Load Data
# Dictionary of columns in data frame
data_dict = {'date': [],
             'animal': [],
             'task': [],             
             'poke1_ts':[],
             'poke2_ts': [],
             'spout1_ts': [],
             'spout2_ts': [],
             'top1_ts': [],
             'poke1_t0':[],
             'poke2_t0': [],
             'spout1_t0': [],
             'spout2_t0': [],
             'top1_t0': []}
# List of dates for analysis
dates= ['090221']
data_path = Path('~/Desktop/mike_bonsai/recordings/').expanduser()
all_paths = []
#Populate Data Dict with meta data and timestamps   
for date in dates:
    for ani in os.listdir(data_path / date): 
        data_paths = list((data_path / date/ ani).rglob('*.csv'))
        if data_paths != []:
            data_dict['date'].append(data_paths[1].name.split('_')[0])
            data_dict['animal'].append(data_paths[1].name.split('_')[1])
            data_dict['task'].append(data_paths[1].name.split('_')[4])
            for ind, csv in enumerate(data_paths):
                data_dict[data_paths[ind].name.split('_')[5] +'_ts'].append(open_time(csv))
                data_dict[data_paths[ind].name.split('_')[5] +'_t0'].append(open_time(csv)[0])
df = pd.DataFrame.from_dict(data_dict)
#df = df.transpose()


#df = pd.DataFrame(data_dict)

In [75]:
#Align to 
##df.poke1_ts = df.poke1_ts - df.top1_t0
#df.poke2_ts = df.poke2_ts - df.top1_t0
#df.spout1_ts = df.spout1_ts - df.top1_t0
#df.spout2_ts = df.spout2_ts - df.top1_t0

In [76]:
df

,date,animal,task,poke1_ts,poke2_ts,spout1_ts,spout2_ts,top1_ts,poke1_t0,poke2_t0,spout1_t0,spout2_t0,top1_t0
0,090221,G6CK3BLT,oa,"[40091.950323, 40169.649497, 40293.092518, 403...","[40114.136166, 40270.604556, 40321.668428, 403...","[40091.950668, 40094.456051, 40169.649868, 401...","[40114.136691, 40116.642688, 40270.604876, 402...","[40085.622323, 40085.638668, 40085.655372, 400...",40091.950323,40114.136166,40091.950668,40114.136691,40085.622323
1,090221,G6CK3BRT,oa,"[43553.127398, 43665.948185, 43829.538956, 439...","[43563.941184, 43817.655667, 43869.941158, 439...","[43553.127872, 43555.639654, 43665.948659, 436...","[43563.941568, 43566.460172, 43817.656179, 438...","[43449.226982, 43449.242585, 43449.259353, 434...",43553.127398,43563.941184,43553.127872,43563.941568,43449.226982


In [77]:
#Only spout on ts 
spout1_on = []
for ind, row in df.iterrows():
    x = row['spout1_ts']
    x = x[0::2]
    spout1_on.append(x)
df['spout1_on'] = spout1_on

In [78]:
#Get all pokes_ts
pokes_ts = []
for ind, row in df.iterrows():
    x = row['poke1_ts'].astype(str).astype(float)
    y = row['poke2_ts'].astype(str).astype(float)
    z = list(itertools.chain(x,y)) 
    z = sorted(z)
    pokes_ts.append(z)
df['pokes_ts'] = pokes_ts

In [79]:
# Number of pokes 
pokes = []
for ind, row in df.iterrows():
    x = row['pokes_ts']
    x=len(x)
    pokes.append(x)
df['poke_num'] = pokes

In [80]:
# Time between pokes in sec
time_between_pokes = []
for ind, row in df.iterrows():
    g = row['pokes_ts']
    g = np.diff(g)
    #g = g[0::2]
    
    time_between_pokes.append(g)
df['time_between_pokes'] = time_between_pokes


In [81]:
#Get 'trials'
## add 
trial_ts = []
for ind, row in df.iterrows():
    x = row['pokes_ts']#.astype(str).astype(float)
    y = [x[i:i+2] for i in range(0, len(x), 2)]
    z =  [x[i+1:i+3] for i in range(0, len(x), 2)]
    t = sorted(y+z)
    trial_ts.append(t)
df['trial_ts']= trial_ts


In [82]:
#Get 'trials #'
trial_num = []
for ind, row in df.iterrows():
    x = row['trial_ts']#.astype(str).astype(float)
    y=len(x)
    trial_num.append(y)
df['trial_num']= trial_num

In [83]:
x = r'C:\Users\nlab\Desktop\mike_bonsai\recordings\090221\G6CK3BLT\oa\090221_G6CK3BLT_control_Rig2_oa_top1DLC_resnet50_object_avoidanceAug31shuffle1_200000.h5'

In [84]:
for ind, row in df.iterrows():
    num_trails = len(row['poke1_ts'])
    df1 = pd.DataFrame([])
    dlc_positions, dlc_labels = open_h5(x)
    for c in range(num_trails):
        df1.at[c, 'first_poke'] = row['poke1_ts'][c]
        df1.at[c, 'second_poke'] = row['poke2_ts'][c]
        df1.at[c, 'first_spout'] = row['spout1_ts'][c]
        df1.at[c, 'second_spout'] = row['spout2_ts'][c]
        time = row['top1_ts']; #time = time[time > df1.loc[c,'first_poke']]; time = time[time < df1.loc[c,'second_poke']]
        df1.at[c, 'trail_timestamps'] = time.astype(object)
        start_stop_inds = (int(np.where([row['top1_ts']==time[0]])[1]), int(np.where([row['top1_ts']==time[-1]])[1]))
        for pos in dlc_positions:
            df1.at[c, pos] = np.array(dlc_positions.loc[start_stop_inds[0]:start_stop_inds[1], pos]).astype(object)
    df1['animal'] = row['animal']; df1['date'] = row['date']; df1['task'] = row['task']

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
x

'C:\\Users\\nlab\\Desktop\\mike_bonsai\\recordings\\090221\\G6CK3BLT\\oa\\090221_G6CK3BLT_control_Rig2_oa_top1DLC_resnet50_object_avoidanceAug31shuffle1_200000.h5'

In [85]:

row =  df.iloc[0]
num_odd_trails = len(row['poke1_ts'])
df1 = pd.DataFrame([])
dlc_positions, dlc_labels = open_h5(x)
count = -1
for c in range(num_odd_trails):
    # odd
    count += 1
    df1.at[count, 'first_poke'] = row['poke1_ts'][c]
    df1.at[count, 'second_poke'] = row['poke2_ts'][c]
    df1.at[count, 'first_spout'] = row['spout1_ts'][c]
    df1.at[count, 'second_spout'] = row['spout2_ts'][c]
    time = row['top1_ts']; time = time[time > df1.loc[count,'first_poke']]; time = time[time < df1.loc[count,'second_poke']]
    df1.at[count, 'trail_timestamps'] = time.astype(object)
    start_stop_inds = (int(np.where([row['top1_ts']==time[0]])[1]), int(np.where([row['top1_ts']==time[-1]])[1]))
    for pos in dlc_positions:
        df1.at[count, pos] = np.array(dlc_positions.loc[start_stop_inds[0]:start_stop_inds[1], pos]).astype(object)
    # even
    count += 1
    if c+1 < len(row['poke1_ts']):
        df1.at[count, 'first_poke'] = row['poke2_ts'][c]
        df1.at[count, 'second_poke'] = row['poke1_ts'][c+1]
        df1.at[count, 'first_spout'] = row['spout2_ts'][c]
        df1.at[count, 'second_spout'] = row['spout1_ts'][c+1]
        time = row['top1_ts']; time = time[time > df1.loc[count,'first_poke']]; time = time[time < df1.loc[count,'second_poke']]
        df1.at[count, 'trail_timestamps'] = time.astype(object)
        start_stop_inds = (int(np.where([row['top1_ts']==time[0]])[1]), int(np.where([row['top1_ts']==time[-1]])[1]))
        for pos in dlc_positions:
            df1.at[count, pos] = np.array(dlc_positions.loc[start_stop_inds[0]:start_stop_inds[1], pos]).astype(object)
df1['animal'] = row['animal']; df1['date'] = row['date']; df1['task'] = row['task']

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
 time = row['top1_ts']; time = time[time > df1.loc[count,'first_poke']]; time = time[time < df1.loc[count,'second_poke']]

In [ ]:
trial_ts

[[[6.327999999994063, 28.513842999993358],
  [28.513842999993358, 84.02717399999528],
  [84.02717399999528, 184.98223299999518],
  [184.98223299999518, 207.47019499999442],
  [207.47019499999442, 236.046104999994],
  [236.046104999994, 277.30385899999965],
  [277.30385899999965, 300.73900799999683],
  [300.73900799999683, 314.0976379999993],
  [314.0976379999993, 421.1633279999951],
  [421.1633279999951, 447.12575999999535],
  [447.12575999999535, 476.5414909999963],
  [476.5414909999963, 497.9078139999983],
  [497.9078139999983, 510.48780799999804],
  [510.48780799999804, 529.2168449999954],
  [529.2168449999954, 593.1598969999977],
  [593.1598969999977, 610.4791809999952],
  [610.4791809999952, 634.0167039999942],
  [634.0167039999942, 652.4089849999946],
  [652.4089849999946, 730.0632699999987],
  [730.0632699999987, 743.7737089999937],
  [743.7737089999937, 802.4784639999998],
  [802.4784639999998, 844.6130689999991],
  [844.6130689999991, 864.4967679999972],
  [864.4967679999972, 

In [ ]:
df1

,first_poke,second_poke,first_spout,second_spout,trial_ts
0,6.328,28.513843,6.328345,28.514368,NaN


In [ ]:
df.poke1_ts[0]

array([40091.950323, 40169.649497, 40293.092518, 40362.926182,
       40399.719961, 40532.748083, 40583.530137, 40614.839168,
       40696.101504, 40738.031308, 40829.396032, 40930.235392,
       40969.052416, 41024.274982, 41171.210137, 41229.320473,
       41259.350784, 41318.153638, 41363.418073, 41435.288153,
       41491.313676, 41519.054003, 41549.973043, 41588.31063 ,
       41634.80823 , 41674.24768 , 41714.104934, 41749.460416,
       41771.932633, 41818.72887 , 41848.198272, 41908.41006 ,
       41953.331033, 42027.994368, 42048.500416, 42077.318131,
       42107.028761, 42131.697049, 42155.488294, 42175.486374,
       42196.644134, 42233.240857, 42252.374873, 42272.159987,
       42306.765721, 42347.11054 , 42369.566502, 42390.642201,
       42410.652633, 42428.262873, 42470.344179, 42493.931008,
       42512.512076, 42533.706163, 42556.41239 , 42594.58254 ,
       42873.914457, 43004.382745, 43150.564377])